In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [16]:

data = pd.read_csv('processed/preprocessed_cv_data.csv')
data['accepted'] = 1

In [17]:
X = data['processed_text'].values
y = data['accepted'].values

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Tokenize the input text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)


In [19]:
# Tokenize the input text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [20]:
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

In [21]:
vocab_size = len(tokenizer.word_index) + 1

# Pad the sequences to ensure equal length
max_sequence_length = max(len(seq) for seq in X_train_tokens)

X_train_pad = pad_sequences(X_train_tokens, maxlen=max_sequence_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_sequence_length, padding='post')


In [22]:
max(len(seq) for seq in X_train_tokens)

2744

In [23]:
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_sequence_length))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [24]:
# Train the model
model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=100, batch_size=32)


Epoch 1/100
6/6 [==============================] - 9s 839ms/step - loss: 0.5290 - accuracy: 0.8012 - val_loss: 0.0764 - val_accuracy: 1.0000
Epoch 2/100
6/6 [==============================] - 4s 649ms/step - loss: 0.0398 - accuracy: 1.0000 - val_loss: 0.0066 - val_accuracy: 1.0000
Epoch 3/100
6/6 [==============================] - 4s 659ms/step - loss: 0.0060 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 4/100
6/6 [==============================] - 4s 661ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 3.6524e-04 - val_accuracy: 1.0000
Epoch 5/100
6/6 [==============================] - 4s 657ms/step - loss: 7.1464e-04 - accuracy: 1.0000 - val_loss: 1.7346e-04 - val_accuracy: 1.0000
Epoch 6/100
6/6 [==============================] - 4s 657ms/step - loss: 2.7098e-04 - accuracy: 1.0000 - val_loss: 1.0899e-04 - val_accuracy: 1.0000
Epoch 7/100
6/6 [==============================] - 4s 672ms/step - loss: 2.2023e-04 - accuracy: 1.0000 - val_loss: 8.1553e-05 - val_ac

In [27]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_pad, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

2/2 [==============================] - 1s 180ms/step - loss: 1.8498e-07 - accuracy: 1.0000
Test Loss: 1.8498306531000708e-07
Test Accuracy: 1.0


In [26]:
model.save('models/model_large.h5')